In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

# set pandas dataframe float precision 
pd.set_option('display.precision',2)


In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
import utils as u
import hv_utils as  hvu


In [ ]:
mro = u.get_mro

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Panel

In [ ]:
from bokeh.plotting import figure

In [ ]:
class Shape(param.Parameterized):
    # parameters
    radius = param.Number(default=1., bounds=(0,1))
    
    # initialization/state managmenet
    # Define other state attributes that are non-parametric
    def __init__(self, **params):
        super().__init__(**params)
        self.figure = figure(x_range=(-1,1), y_range=(-1,1))
        self.renderer = self.figure.line(*self._get_coords())
    def _get_coords(self):
        return [],[]
    def view(self):
        return self.figure
    
class Circle(Shape):
    n = param.Integer(default=100, precedence=-1) 
    # precedence < pn.Param.display_threshold will not be `widgetized`
    
    def __init__(self, **params):
        super().__init__(**params)
        
    @param.depends('n', 'radius')
    def _get_coords(self):
        angles = np.linspace(0, 2*np.pi, num=self.n+1)
        return (self.radius*np.sin(angles),
                self.radius*np.cos(angles))
    
    @param.depends('radius', watch=True)
    def update(self):
        xs,ys= self._get_coords()
        self.renderer.data_source.data.update({'x': xs, 'y':ys})
        
class NGon(Circle):
    n = param.Integer(default=3, bounds=(3,10),
                      precedence=1)
    
    @param.depends('radius', 'n', watch=True)
    def update(self):
        xs,ys = self._get_coords()
        self.renderer.data_source.data.update({'x': xs, 'y': ys})
        

In [ ]:
hexagon = NGon(n=6, radius=0.3)
show(hexagon.view())

In [ ]:
c,show(c.view())

In [ ]:
# c.set_param(n=10)
# show(c.view())
c.set_param(radius=0.5)
show(c.view())

In [ ]:
shapes = [hexagon, c]
class ShapeViewer(param.Parameterized):
    
    # parameter
    shape = param.ObjectSelector(default=shapes[0], objects=shapes)
    
    @param.depends('shape')
    def view(self):
        return self.shape.view()
    
    @param.depends('shape')
    def title(self):
        return f"{type(self.shape).__name__}, r={self.shape.radius}"
    
    def panel(self):
        return pn.Column(self.title(), self.view())

In [ ]:
sv = ShapeViewer()
pn.Row(sv.param, sv.panel())

---
## Panel Pipelines
Modified: Jun 25, 2019

In [ ]:
pn.extension('katex')

pipeline = pn.pipeline.Pipeline()

class Stage1(param.Parameterized):
    a = param.Number(default=5, bounds=(0,10))
    b = param.Number(default=5, bounds=(0,10))
    
    @param.output(c=param.Number, d=param.Number)
    def output(self):
        return self.a*self.b, self.a**self.b
    
#     @param.depends('a','b')
    def view(self):
        c,d = self.output()
        return pn.pane.LaTeX(f"""
        ${self.a} * {self.b} = {c}$, 
        ${self.a} ** {self.b} = {d}$
        """)
    def panel(self):
        return pn.Row(self.param, self.view())
    
s1 = Stage1()
s1.panel()
        

In [ ]:
s1.param.outputs()

In [ ]:
s1.panel()


In [ ]:
pipeline.add_stage('Stage1',s1)

In [ ]:
## Second stage
class Stage2(param.Parameterized):
    c = param.Number(default=5, bounds=(0,None), precedence=-1)
    exp = param.Number(default=0.1, bounds=(0,3))
    
#     @param.depends('c','exp')
    def view(self):
        return pn.pane.LaTeX(f"""
        ${self.c}**{self.exp} = {self.c**self.exp}$
        """)
    
    def panel(self):
        return pn.Row(self.param, self.view)
s2 = Stage2()
s2.panel()

In [ ]:
pipeline.add_stage('Stage2', s2)

In [ ]:
pipeline.layout

---
## Bokeh basics
Modified: Jun 25, 2019

In [ ]:
from bokeh.plotting import figure, output_file, output_notebook, show
# 1. Define the output destination
output_notebook()

In [ ]:
# 2. Prepare data
x = [1,2,3,4,5]
y = [3,2,1,4,5]

# 3. Configure the settings for visualization
##  call `figure`
p = figure(title='bokeh_test', 
           x_axis_label='x',
           y_axis_label='y')
##  Add renders for your glyphs
p.line(x,y, legend='temperature', line_width=2
      )

## Serve the result plot(aka. figure) using eg `show` or `save
show(p)

In [ ]:
# data2
N = 100
x = np.random.random(size=N) *10
y = np.random.random(size=N)*10
radii = np.random.random(size=N) *1.5
colors = [f"#{int(r):02x}{int(g):02x}{int(150):02x}" 
          for (r,g) in zip(20*x+50, 20*y+50)]
TOOLS = 'crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select'

p = figure(tools=TOOLS, x_range=(0,10), y_range=(0,10))
p.circle(x,y, radius=radii, fill_color=colors, fill_alpha=0.5)

show(p)

In [ ]:
# example 3
from bokeh.layouts import gridplot
# data
n = 100
xs = np.linspace(-2*np.pi, 2*np.pi, num=n)
ys1 = np.sin(xs)
ys2 = np.cos(xs)

# get plots/figures
f1 = figure(width=200, height=200)
f2 = figure(width=200, height=200,
            x_range=f1.x_range)#, y_range=f1.y_range) # links x and y ranges for linked panning

In [ ]:
f1.x_range, f2.x_range

In [ ]:
# add renders to the plot
f1.line(xs,ys1,color='yellow',legend='sin')
f2.line(xs,ys2,color='red',legend='cos')

In [ ]:
# show results
layout = gridplot([[f1,f2]])

In [ ]:
show(layout)

## Linked brushing
a selection on one plot causes a selection to update on the other plots
- Done by sharing a `ColumnDataSource` between two figures


In [ ]:
# NEW: create a column data source for the plots to share
from bokeh.models import ColumnDataSource

In [ ]:
# source = ColumnDataSource(

In [ ]:
data = {'col1': [1,2,3,4], 'col2': np.array([10.0, 20.0, 30.0, 40.0])}
# data = dict(x=xs, y1=ys1, y2=ys2)
source = ColumnDataSource(data)
source, #data

In [ ]:
id(source.data), id(data), (source.data is data)

In [ ]:
source.to_df().head()

- Create a new plot and add a renderer for your glyph with a constructor that takes in the source (of `ColumnDataSource` instance) and its column names to select `x` and `y` data


In [ ]:
p1 = figure(width=300, height=300, tools=TOOLS)
p2 = figure(width=300, height=300, tools=TOOLS)
p1.circle(x='x', y='y1',source=source)
p2.circle(x='x', y='y2',source=source)

show(gridplot([[p1,p2]]))

In [ ]:
p3 = figure(width=500, height=300, x_axis_type='datetime')
xs = pd.date_range('2019-04-01', '2019-04-30', freq='D')
x_ind = np.linspace(0,2*np.pi,len(xs))
x_ind

In [ ]:
y1 = np.sin(x_ind)
p3.circle(x=xs, y=y1)
show(p3)

In [ ]:
p3.xaxis.axis_label='Date'
p3.yaxis.axis_label='yval'

In [ ]:
show(p3)

---
## Back to FLDAS
Modified: Jun 25, 2019

Combining holoviews objects with Bokeh models for custome interactive visualization


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 500; H_IMG = 500
W_PLOT = 300; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

In [ ]:
import inspect

trange = list(map(pd.Timestamp, hvd_ea.range('time')))

class ZonalExplorer(param.Parameterized):
    
    ################################################################################
    ## Parameters
    ################################################################################
    print("Initializing parameter...")
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10],objects=varnames)
    time_slider= param.Date(trange[0], bounds=trange)

    
    ################################################################################
    ## Computations
    ################################################################################
    def __init__(self, **params):
        print("Called: ", inspect.currentframe().f_code.co_name, 'from ', inspect.currentframe().f_back.f_code.co_name)

        super().__init__(**params)
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa
        self.time_values = self.xrd.get_index('time')
        self._set_empty_tplot()
        
    @param.depends('region', 'varname', 'time_slider', watch=True)
    def dyn_timg(self): #update_timg
        print("Called: ", inspect.currentframe().f_code.co_name, 'from ', inspect.currentframe().f_back.f_code.co_name)

        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa
        self.time_values = self.xrd.get_index('time')
        data = self.xrd[self.varname].sel(time=self.time_slider)
        hvimg = gv.Image(data, ['X','Y'], self.varname,
                         crs=ccrs.PlateCarree(),
                         label=self.varname).opts(**img_opts)
        self.timg = timg
        
        return basemap * datashade(self.timg, **ds_opts)
    
    @param.depends('varname') #no need to be updated ie. watching `varname` parameter
    def _set_empty_tplot(self):
        print("Called: ", inspect.currentframe().f_code.co_name, 'from ', inspect.currentframe().f_back.f_code.co_name)

        self.empty_tplot = hv.Curve( 
            (self.time_values, np.empty(len(self.time_values))), 
            'time', self.varname).opts(line_alpha=0.)
        return self.empty_tplot
    
    @param.depends('time_slider', watch=True)
    def dyn_vline(self):
        #todo
        # 1. set datetime axes for holoviews 
        ## set an empty plot and add the vline there
        print('type of timeslider val: ', type(self.time_slider))
        print("Called: ", inspect.currentframe().f_code.co_name, 'from ', inspect.currentframe().f_back.f_code.co_name)
        
        self.vline = self.empty_tplot * hv.VLine(self.time_slider).opts(color='black')

        return self.vline

        
    def panel(self):
        return pn.Row(self.param, self.dyn_timg(), self.dyn_vline())
    
    

In [ ]:
ze = ZonalExplorer()
# ze.empty_tplot * hv.VLine(ze.time_values[10]).opts(color='black')
# ze.panel().servable()

In [ ]:
z = ze.empty_tplot

In [ ]:
# todo: change 

In [ ]:
temp_img = ze.timg()
temp_pimg = renderer.get_plot(temp_img)
temp_ds = datashade(temp_img)
temp_pds = renderer.get_plot(temp_ds)
print(temp_img, temp_ds, id(temp_img), id(temp_ds))
print(temp_pimg, temp_pds, id(temp_pimg), id(temp_pds))





In [ ]:
print(temp_img)

---
## Dummy Explorer
Modified: Jun 26, 2019



In [ ]:
from bokeh.sampledata.iris import flowers
from bokeh.sampledata.autompg import autompg_clean
from bokeh.sampledata.population import data

In [ ]:
class Dummy(param.Parameterized):
    title = param.String(default="Data summary")
    dataset = param.ObjectSelector(default='iris', objects=['iris', 'autompg', 'population'])
    rows = param.Integer(default=10, bounds=(0, 100))

    def data(self):
        if self.dataset == 'iris':
            return flowers
        elif self.dataset == 'autompg':
            return autompg_clean
        else:
            return data

---
## Experiment: DynamicMap and its HV Plot object
With `param`'s parameter and \@param.depends

In [ ]:
class Dummy(param.Parameterized):
    # parameter
    n = param.Integer(default=3)
    
    # Initialization
    def __init__(self, **params):
        super().__init__(**params)
#         init_dmap()
        
    # computations
    def get_points_static(self):
        return hv.Points(np.arange(4)).opts(color='red')
    
    @param.depends('n')
    def get_points(self):
        ys = np.arange(self.n)
#         print("Is it updating the dmap_fig/plot or just the image?")
#         print('\tdmap_fig: ', self.dmap_fig)
        return hv.Points(ys)
    
#     @param.depends
    def dmap(self):
        self.dmap = hv.DynamicMap(self.get_points)
        self.dmap_fig = renderer.get_plot(self.dmap)
        print('dmap_fig: ', self.dmap_fig)
        pdb.set_trace()
        return self.dmap
        
    def view(self):
        return self.get_points()
                                  
    def panel(self):
        return pn.Row(self.param, self.view)

In [ ]:
dummy = Dummy()


In [ ]:
dummy.dmap()


In [ ]:
mro(pn.panel(dummy.dmap)), display(pn.panel(dummy.dmap))

In [ ]:
mro(dummy.get_points_static())

In [ ]:
static_method_panel = pn.panel(dummy.get_points_static)
mro(static_method_panel)

In [ ]:
dependent_method_panel = pn.panel(dummy.get_points)
u.nprint(mro(dependent_method_panel))
u.nprint(dependent_method_panel._callbacks)

In [ ]:
dmap_panel = pn.panel(dummy.dmap)
mro(dmap_panel)

In [ ]:
static_method_panel.object
# mro(static_method_panel.get_root())

In [ ]:
static_method_panel._callbacks

In [ ]:
print(dmap_panel.object)

In [ ]:
p = dummy.panel()

In [ ]:
p

In [ ]:
row = pn.Row(dummy.view)

In [ ]:
type(row)


In [ ]:
row.objects

In [ ]:
dummy.view, mro(dummy.view)



In [ ]:
r2 = pn.Row(dummy.param, dummy.view)

In [ ]:
r2

In [ ]:
r2.objects


In [ ]:
pn.panel(dummy.get_points)

In [ ]:
mro(pn.panel(dummy.get_points))

In [ ]:
pn.panel(dummy.get_points())

In [ ]:
mro(pn.panel(dummy.get_points()))

---
## Clean BoxEdit from Scratch
Modified: Jun 26, 2019

- Incoporating HoloMap and DynamicMap objects
    - [src](http://holoviews.org/user_guide/Live_Data.html)

In [ ]:
xrd=xrd_ea
varname=varnames[7]
t = '2019-04-10'
time_values = xrd.get_index('time')
# time_values = pd.date_range('2019-04-01', '2019-04-30', freq='D')

data = xrd[varname]#.sel(time=t)
hvd = gv.Dataset(data, ['X','Y','time'], varname, crs=ccrs.PlateCarree())

def get_timg(time):
    data = xrd[varname].sel(time=time)
    return gv.Image(data, ['X','Y'], varname, crs=ccrs.PlateCarree())
dmap_timg = datashade(hv.DynamicMap(get_timg, kdims='time'),
                      **ds_opts)

In [ ]:
# dmap_realized = dmap_timg.redim.values(time=time_values)
# pn.panel(dmap_realized)

In [ ]:
# Set BoxEdit stream
polys = gv.Polygons([], crs=ccrs.PlateCarree())
box_stream = streams.BoxEdit(source=polys)


In [ ]:
def get_empty_tplot():
    # Set empty tplot and vlines
    dummy_df = pd.DataFrame({'time': time_values, 
                             varname:np.zeros(len(time_values))})
    empty_tplot= hv.Curve(dummy_df, 'time', varname).opts(line_alpha=0.)
    return empty_tplot

def get_hvmap_vlines(vline_opts={}):
    # Set empty tplot and vlines
    empty_tplot= get_empty_tplot()
    vlines = hv.HoloMap({t: hv.VLine(t).opts(**vline_opts) for t in time_values},
                        kdims='time')
#     return empty_tplot * vlines
    return vlines

empty_tplot = get_empty_tplot()
vlines = get_hvmap_vlines(dict(color='green')) 
print(vlines)

In [ ]:
# pn.panel(dmap_timg + hvmap_vlines)

Success!!!

In [ ]:
def roi_curves(data):
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: empty_tplot})
    curves = {}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0,x1,y0,y1) in enumerate(data):
        selection = hv.Dataset( xrd[varname].sel(X=slice(x0,x1), Y=slice(y0,y1)), kdims=['X','Y','time'])
        curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', varname)
    return hv.NdOverlay(curves, label='roi_curves', kdims='roi')

dmap_roi_curves = hv.DynamicMap(roi_curves, streams=[box_stream])

In [ ]:
print(dmap_roi_curves)
dmap_roi_curves

In [ ]:
comp1 = dmap_timg*polys + vlines*empty_tplot
print(comp1)
print(dmap_roi_curves)

In [ ]:
empty_tplot * dmap_roi_curves 

In [ ]:
print(vlines)

In [ ]:
test_rois = {'x0': [28.484264530634945, 23.75232112846949], 
             'y0': [5.161857571360065, -4.790934173611567], 
             'x1': [39.80125756910407, 35.76445237820839], 
             'y1': [16.341892860897076, 10.938420208725127]}
# test_rois
box_stream.event(data=test_rois)
# box_stream.event(data={})

In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))
class Time(param.Parameterized):
    
    time= param.Date(trange[0], bounds=trange)
    

In [ ]:
# time stream
tparam = Time()
tparam
stream = streams.Params(tparam)
twidget =pn.panel(tparam.param
         , widgets={'time': pn.widgets.DateSlider})

In [ ]:
## better way

from holoviews.streams import Stream, param
# Time = Stream.define('Time', t=0.0)
Time = Stream.define('Time', time=param.Date(trange[0], bounds=trange, doc='a time parameter'))
tparam = Time()
print(tparam)
hv.help(Time)

In [ ]:
u.nprint(tparam, twidget.objects, stream)

In [ ]:
dmap_debug = hv.DynamicMap( hvu.get_debug_div, streams=[tparam])

In [ ]:
dmap_vline = hv.DynamicMap(lambda time: hv.VLine(time), streams=[tparam])

In [ ]:
dmap_img = hv.DynamicMap(lambda time: get_timg(time), streams=[tparam])
dmap = dmap_img * polys

In [ ]:
stream.event(time=pd.Timestamp('2019-04-12'))
stream

In [ ]:
pn.Row(tparam, dmap_img + dmap_debug).servable()

In [ ]:
tparam.event(time=pd.Timestamp('2019-04-05'))

In [ ]:
dmap_img.streams, dmap_debug.streams

In [ ]:
dmap_debug

In [ ]:
pn.panel(tparam.param
         , widgets={'time': pn.widgets.DateSlider})

In [ ]:
stream.tr

In [ ]:
tparam

In [ ]:
mro(tparam)

In [ ]:
tparam